In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn import model_selection, preprocessing, metrics
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from utils import classification_util as cu
from nn_framework import *

## Load Data

In [62]:
data_path = r'/home/david/TU/ML/Loan/Data/loan-10k.lrn.csv'
df = pd.read_csv(data_path)
df

,ID,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,...,debt_settlement_flag,issue_d_month,issue_d_year,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_pymnt_d_year,last_credit_pull_d_month,last_credit_pull_d_year,grade
0,24341,12500.0,12500.0,12500.0,36 months,7.21,387.17,< 1 year,MORTGAGE,81000.0,...,N,6,2018,6,2000,2,2019,2,2019,A
1,67534,33850.0,33850.0,33775.0,60 months,20.99,915.57,1 year,MORTGAGE,80000.0,...,N,10,2015,9,1984,2,2019,2,2019,E
2,35080,10000.0,10000.0,10000.0,60 months,20.00,264.94,< 1 year,RENT,36580.0,...,N,9,2017,10,2006,1,2018,11,2018,D
3,4828,20250.0,20250.0,20250.0,36 months,14.31,695.15,9 years,RENT,48700.0,...,N,0,2015,6,1996,6,2016,9,2017,C
4,59259,25000.0,25000.0,25000.0,36 months,14.99,866.52,1 year,MORTGAGE,85000.0,...,N,11,2016,0,2002,2,2019,2,2019,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,6644,20000.0,20000.0,20000.0,60 months,16.02,486.58,5 years,OWN,54000.0,...,N,8,2017,11,1991,2,2019,2,2019,C
9996,25910,7500.0,7500.0,7500.0,36 months,9.49,240.22,10+ years,RENT,46386.0,...,N,10,2014,4,2004,3,2017,6,2018,B
9997,95698,20750.0,20750.0,20750.0,60 months,15.05,494.19,< 1 year,RENT,185000.0,...,N,4,2017,2,2004,2,2019,2,2019,C
9998,27371,11000.0,11000.0,11000.0,60 months,16.29,269.20,1 year,RENT,38500.0,...,N,9,2014,8,2006,3,2016,10,2016,D


## Experiments

In [63]:
def transformer(X):
        log_tranform_col = ['int_rate', 'annual_inc', 'total_rev_hi_lim', 'tot_hi_cred_lim', 'total_bc_limit']
        c_root_tranform_col = ['installment', 'total_acc', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 
                            'total_il_high_credit_limit', 'total_bal_ex_mort', 'avg_cur_bal', 'bc_open_to_buy', 
                            'revol_bal', 'total_rec_int', 'last_pymnt_amnt', 'tot_coll_amt','tot_cur_bal']

        for col in c_root_tranform_col:
            X[col] = np.cbrt(X[col])
        for col in log_tranform_col:
            X[col] = np.log(X[col])
        return X

In [64]:
def train_test(X, y):
    X_tr, X_te, y_tr, y_te = train_test_split(X,y, random_state=40, stratify=y, test_size=0.2)
    X_te_id = X_te.pop('ID')
    X_tr_id = X_tr.pop('ID')
    return X_tr, X_te, y_tr, y_te, X_te_id, X_tr_id

In [65]:
nn = NNFramework()
data = df.copy()

In [66]:
y = data.pop('grade')
# X = data.pop('addr_state')
X = data
X = transformer(X)
cols = list(X.select_dtypes('object').columns)
nn.fit_encoder(X, cols)
X = nn.encode_dataset(X)
X_tr, X_te, y_tr, y_te, X_te_id, X_tr_id = train_test(X,y)

In [67]:
random_seed = 1038
scaling = True
oversampling = True

scaler = preprocessing.StandardScaler() if scaling else None

In [68]:
# activation_functions = ['relu', 'sigmoid']
# learning_rates = [0.001, 0.01, 0.1]
# hidden_layer_sizes = [(5,), (32,), (16, 16), (10, 5, 5), (16, 8, 8), (64, 32, 32),]

activation_functions = ['relu']
learning_rates = [0.0001]
hidden_layer_sizes = [(5,), (32,), (16, 16),]

In [69]:
methods = []

for af in activation_functions:
    for lr in learning_rates:
        for hls in hidden_layer_sizes:
            methods.append((f'MLP-{af}-{lr}-{hls}', MLP(n_iter=1000, activation_function=af, learning_rate=lr, hidden_layer_sizes=hls)))
    
pipelines = cu.define_pipelines(methods, scaler=scaler, oversampling=oversampling)



In [70]:
import warnings
warnings.filterwarnings("ignore")

In [71]:
cv_num = 5
model_params = {}
models = {}

for model_name, pipeline in pipelines.items():
    cv_results = cross_validate(pipeline, X, y, cv=cv_num, scoring='f1_macro', return_estimator=True, n_jobs=5)

    models[model_name] = cv_results['estimator']
    model_params[model_name] = {}

    num_cols = ['test_score', 'fit_time', 'score_time']

    for num_col in num_cols:
        model_params[model_name][num_col] = cv_results[num_col]
        model_params[model_name][f'{num_col}_mean'] = cv_results[num_col].mean()
        model_params[model_name][f'{num_col}_std'] = cv_results[num_col].std()
    
    model_params[model_name]['parameter_num'] = cv_results['estimator'][0][model_name].number_of_params_
    model_params[model_name]['hidden_layer_sizes'] = cv_results['estimator'][0][model_name].hidden_layer_sizes
    model_params[model_name]['activation_function'] = cv_results['estimator'][0][model_name].activation_function
    model_params[model_name]['learning_rate'] = cv_results['estimator'][0][model_name].learning_rate
    model_params[model_name]['converged'] = [e[model_name].converged_ for e in cv_results['estimator']]
    model_params[model_name]['validation_losses'] = [e[model_name].validation_losses_ for e in cv_results['estimator']]
    model_params[model_name]['training_losses'] = [e[model_name].training_losses_ for e in cv_results['estimator']]
    model_params[model_name]['num_iter'] = np.array(list([len(e[model_name].training_losses_) for e in cv_results['estimator']])).mean()

    print(model_name)
    print(
        f"f1 scores: {model_params[model_name]['test_score']}\n" +
        f"f1 mean: {model_params[model_name]['test_score_mean']:.3f}\n" +
        f"f1 std: {model_params[model_name]['test_score_std']:.3f}\n"
    )
    print('----------------------------------------------------------------------------------------------------')
    

/home/david/TU/ML/ml_ss23_group13/Ex2/MLP.py:188: RuntimeWarning: divide by zero encountered in log
  loss -= (idx == y)*np.log(y_prob)
/home/david/TU/ML/ml_ss23_group13/Ex2/MLP.py:188: RuntimeWarning: invalid value encountered in multiply
  loss -= (idx == y)*np.log(y_prob)


In [89]:
df_param = pd.DataFrame(model_params).transpose()
df_param = df_param.reset_index(drop=False)
df_param = df_param.rename(columns={'index': 'model'})

In [90]:
df_param

,model,test_score,test_score_mean,test_score_std,fit_time,fit_time_mean,fit_time_std,score_time,score_time_mean,score_time_std,parameter_num,hidden_layer_sizes,activation_function,learning_rate,converged,validation_losses,training_losses,num_iter
0,"MLP-sigmoid-0.1-(5,)","[0.4362852044060047, 0.4876745255654208, 0.428...",0.471599,0.033024,"[93.93587517738342, 46.835485219955444, 84.829...",77.885424,16.43483,"[0.020564794540405273, 0.02417612075805664, 0....",0.019963,0.003435,912,"(5,)",sigmoid,0.1,"[True, True, True, True, True]","[[1.1131053327911047, 1.0406877704819701, 1.01...","[[1.0990141234976365, 1.0059124858953366, 0.96...",41.0
1,"MLP-sigmoid-0.1-(32,)","[0.5349988795469858, 0.49149372563813604, 0.51...",0.509671,0.020828,"[60.83073973655701, 53.84045195579529, 50.0145...",57.047117,7.081147,"[0.01794123649597168, 0.019283533096313477, 0....",0.018587,0.001212,5799,"(32,)",sigmoid,0.1,"[True, True, True, True, True]","[[0.9951244575822593, 0.6790727588415476, 0.49...","[[0.9847171097314604, 0.6245553389697279, 0.43...",25.0
2,"MLP-sigmoid-0.1-(16, 16)","[0.43677654898762563, 0.4595357776617671, 0.39...",0.474968,0.062471,"[54.44634819030762, 111.57658672332764, 98.937...",106.017389,29.309367,"[0.0181882381439209, 0.017653703689575195, 0.0...",0.018683,0.000646,3175,"(16, 16)",sigmoid,0.1,"[True, True, True, True, True]","[[1.4004516858409055, 1.0942301915307033, 1.08...","[[1.3937962514305133, 1.0876466687875868, 1.08...",39.2
3,"MLP-sigmoid-0.1-(10, 5, 5)","[0.0015628329899836611, 0.0017040613462084634,...",0.001619,0.000069,"[42.428252935409546, 41.61358428001404, 37.060...",40.774826,1.919014,"[0.01812577247619629, 0.019957542419433594, 0....",0.019292,0.001816,1867,"(10, 5, 5)",sigmoid,0.1,"[True, True, True, True, True]","[[2.1328512561870294, 2.1332926767628844, 2.13...","[[2.1326713334566954, 2.133114691675851, 2.133...",11.0
4,"MLP-sigmoid-0.1-(16, 8, 8)","[0.0015628329899836611, 0.0017040613462084634,...",0.001619,0.000069,"[43.7005033493042, 46.58681106567383, 45.71397...",45.022591,1.324975,"[0.018641233444213867, 0.018329381942749023, 0...",0.019168,0.001294,3055,"(16, 8, 8)",sigmoid,0.1,"[True, True, True, True, True]","[[2.1838081753208565, 2.183953753241708, 2.184...","[[2.1836521310328796, 2.1837940762391104, 2.18...",11.0
5,"MLP-sigmoid-0.1-(64, 32, 32)","[0.06397250401906979, 0.06388642413487133, 0.0...",0.063904,0.000034,"[53.13920545578003, 56.13001465797424, 57.0656...",55.244585,1.668365,"[0.029093265533447266, 0.02825140953063965, 0....",0.027441,0.005096,14503,"(64, 32, 32)",sigmoid,0.1,"[True, True, True, True, True]","[[2.6004871441736848, 2.600487428089203, 2.600...","[[2.6008490514167106, 2.6008492815641473, 2.60...",11.0


In [ ]:
df_param.to_csv(r'results/fertility_params.csv', index=False)